In [ ]:
# imports
from zipfile import ZipFile
from urllib.request import urlretrieve
import os
import pickle
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
# download-paths
urls = ["https://image-net.org/data/downsample/Imagenet64_train_part1.zip", 
        "https://image-net.org/data/downsample/Imagenet64_train_part2.zip",
        "https://image-net.org/data/downsample/Imagenet64_val.zip"]

root = "Data/"

In [ ]:
# download data into root, unpack and delete zips
for url in urls:
    fileending = url.rsplit('/',1)[-1]
    print("Start dowloading {}".format(fileending))
    urlretrieve(url, root + fileending)
    print ("Start unpacking")
    with ZipFile(root + fileending, 'r') as zipObj:
        zipObj.extractall(root)
    print ("Finished unpacking")
    print ("Delete Zip")
    try:
        os.remove(root + fileending)
        print("Delete Succesfull")
    except OSError as e:
        print ("Error: %s - %s." % (e.filename, e.strerror))             

In [ ]:
# read routines
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo)
    return dict

def load_data(input_file, size):

    d = unpickle(input_file)
    x = d['data']
    y = d['labels']
    
    dim = x.shape[1] // 3
    x = np.dstack((x[:, :dim], x[:, dim:2*dim], x[:, 2*dim:]))
    x = x.reshape((x.shape[0], size[0], size[1], 3))

    return x, y

In [ ]:
# write data into tfrecords
from src import utils
import tensorflow as tf

write_root = "TFRecords/ImageNet64_Train/imagenet64train_"
filepaths = sorted(os.listdir(root))
filepaths_train = sorted(filepaths[:-1], key=lambda x: int(x.rsplit('_',1)[-1]))
filepaths_val = filepaths[-1]

with open('map_clsloc.txt', 'r', encoding='utf8') as f:
    classes = f.readlines()
    classes = [c.split()[-1] for c in classes]

for nr,filepath in enumerate(filepaths_train):
    x, y = load_data(root + filepath, (64,64))
    y = [i-1 for i in y]
    #x = ((x / np.float32(255)) * 2) - 1.0
    write_path = write_root + str(nr) + ".tfrecord"
    writer = utils.TfPartWriter(write_path, x.shape[0])
    
    for frame, label in zip(x,y):
        datadict = {
                'frame': utils.bytes_feature([frame.tostring()]),
                'class' : utils.bytes_feature([classes[label].encode()]),
                'nr': utils.int_feature([label])
                }

        example = tf.train.Example(features = tf.train.Features(feature = datadict))
        writer.write_part(example)
    

In [ ]:
# Necessary imports
import sys
sys.path.append('src/')
import utils, models, layers
import os
import pickle
import numpy as np

reader = utils.DataReader(64, "TFRecords/ImageNet64_Train")
for nr, batch in enumerate(reader.data):
    print(nr)
    
